In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np

In [72]:
from CNN_torch import SpatialAttention

x = torch.normal(0, 1, (1, 1, 20, 1))
sa = SpatialAttention()
sa(x).shape

torch.Size([1, 1, 20, 1])

In [84]:
x = torch.normal(0, 1, (1, 1, 20, 1))
conv = torch.nn.Conv2d(in_channels=1, out_channels=32, kernel_size=(3, 1), stride=1, padding=1, bias=False)
conv(x).shape

torch.Size([1, 32, 20, 3])

In [77]:
x = torch.normal(0, 1, (1, 8, 20, 3))
b, c, h, w = x.shape
split_s_h = h // 2
split_s_w = w // 2
xs = torch.cat(torch.split(x[:, :c // 4], split_s_h, dim=-2), dim=1).contiguous()
xs = torch.cat(torch.split(xs, split_s_w, dim=-1),
                           dim=1).contiguous()
xs.shape

torch.Size([1, 12, 10, 1])

In [82]:
torch.cat(torch.split(x[:, :c // 4], split_s_h, dim=-2), dim=1).contiguous().shape

torch.Size([1, 4, 10, 3])

In [11]:
N, C, H, W = (1, 3, 4, 4)
x = torch.tensor([float(i) for i in range(1 * 3 * 4 * 4)]).reshape((1, 3, 4, 4))
y = torch.fft.rfft2(x, norm='ortho')
y_r, y_i = y.real, y.imag
print(x.shape)
print(y.shape)

torch.Size([1, 3, 4, 4])
torch.Size([1, 3, 4, 3])


In [4]:
x

tensor([[[[ 0.,  1.,  2.,  3.],
          [ 4.,  5.,  6.,  7.],
          [ 8.,  9., 10., 11.],
          [12., 13., 14., 15.]],

         [[16., 17., 18., 19.],
          [20., 21., 22., 23.],
          [24., 25., 26., 27.],
          [28., 29., 30., 31.]],

         [[32., 33., 34., 35.],
          [36., 37., 38., 39.],
          [40., 41., 42., 43.],
          [44., 45., 46., 47.]]]])

In [5]:
y.real.shape, y.imag.shape

(torch.Size([1, 3, 4, 3]), torch.Size([1, 3, 4, 3]))

In [18]:
y.size()

torch.Size([1, 3, 4, 3])

In [68]:

# 迁移过来总感觉有些问题……
def FU(x, out_channels):
    batch_size, in_channels, height, width = x.shape
    r_size = x.shape

    conv = torch.nn.Conv2d(in_channels=in_channels * 2, out_channels=out_channels * 2, kernel_size=1, stride=1,
                           padding=0, bias=False)
    # init param
    conv.weight.data = torch.tensor([float(i) for i in range(out_channels * 2 * in_channels * 2 * 1 * 1)]).reshape(
        (out_channels * 2, in_channels * 2, 1, 1))

    bn = torch.nn.BatchNorm2d(out_channels * 2)
    relu = torch.nn.ReLU(inplace=True)

    y = torch.fft.rfft2(x, norm='ortho')
    y_r, y_i = y.real, y.imag
    y = torch.concat((y_r, y_i), dim=1)
    y = conv(y)
    y = bn(y)
    y = relu(y)
    y_r, y_i = torch.split(y, out_channels, dim=1)
    y = torch.stack((y_r, y_i), dim=4)
    y = torch.view_as_complex(y)
    # y = y.view((batch_size, -1, 2,) + y.size()[2:]).permute(0, 1, 3, 4, 2).contiguous()
    # z = torch.fft.irfft2(torch.view_as_complex(y), s=r_size[2:], norm='ortho')
    z = torch.fft.irfft2(y, s=r_size[2:], norm='ortho')
    print(z.shape)
    return z


z = FU(x, 6)
z, z.shape

torch.Size([1, 6, 4, 4])


(tensor([[[[0.8686, 0.8686, 0.8686, 0.8686],
           [0.7808, 0.7808, 0.7808, 0.7808],
           [0.6930, 0.6930, 0.6930, 0.6930],
           [0.7808, 0.7808, 0.7808, 0.7808]],
 
          [[0.8124, 0.8124, 0.8124, 0.8124],
           [0.8124, 0.8124, 0.8124, 0.8124],
           [0.8124, 0.8124, 0.8124, 0.8124],
           [0.8124, 0.8124, 0.8124, 0.8124]],
 
          [[0.8145, 0.8145, 0.8145, 0.8145],
           [0.8145, 0.8145, 0.8145, 0.8145],
           [0.8145, 0.8145, 0.8145, 0.8145],
           [0.8145, 0.8145, 0.8145, 0.8145]],
 
          [[0.8152, 0.8152, 0.8152, 0.8152],
           [0.8152, 0.8152, 0.8152, 0.8152],
           [0.8152, 0.8152, 0.8152, 0.8152],
           [0.8152, 0.8152, 0.8152, 0.8152]],
 
          [[0.8156, 0.8156, 0.8156, 0.8156],
           [0.8156, 0.8156, 0.8156, 0.8156],
           [0.8156, 0.8156, 0.8156, 0.8156],
           [0.8156, 0.8156, 0.8156, 0.8156]],
 
          [[0.8158, 0.8158, 0.8158, 0.8158],
           [0.8158, 0.8158, 0.8158, 0.81